In [2]:
import json
import pandas as pd
import numpy as np


pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [15]:
def wing_vol_curve_with_smoothing(
    strikes,
    F,              # Forward price used to define log-moneyness
    vr,             # Volatility reference at central skew point
    sr,             # Slope reference at central skew point
    pc,             # Put curvature: bend on put wing (left of ATM)
    cc,             # Call curvature: bend on call wing (right of ATM)
    dc,             # Down cutoff (log-moneyness, negative value)
    uc,             # Up cutoff (log-moneyness, positive value)
    dsm=0.5,        # Down smoothing range beyond dc
    usm=0.5,        # Up smoothing range beyond uc
    VCR=0.0,        # Volatility change rate w.r.t forward move
    SCR=0.0,        # Slope change rate w.r.t forward move
    SSR=100,        # Skew swimmingness rate (0 = fixed, 100 = ATM follows F)
    Ref=None,       # Reference price (for fixed skew)
    ATM=None        # ATM forward price (for floating skew)
):
    """
    Generate a volatility curve using the Orc Wing Model with parabolic smoothing.

    This version reproduces Orc's exact smoothing style:
    - Parabolic skew on both put and call wings.
    - Smooth cubic (Hermite-style) interpolation in the smoothing zones.
    - Flat extrapolation beyond the smoothing cutoffs.

    Parameters:
        strikes (array): Strike prices to compute volatility for.
        F (float): Forward price for normalization (used to compute log-moneyness).
        vr (float): Volatility reference at the central skew point.
        sr (float): Slope reference at the central skew point.
        pc (float): Curvature for the put wing (left of ATM).
        cc (float): Curvature for the call wing (right of ATM).
        dc (float): Down cutoff in log-moneyness where put wing ends.
        uc (float): Up cutoff in log-moneyness where call wing ends.
        dsm (float): Down smoothing width beyond dc (default 0.5).
        usm (float): Up smoothing width beyond uc (default 0.5).
        VCR (float): Volatility change rate when forward deviates from reference.
        SCR (float): Slope change rate when forward deviates from reference.
        SSR (float): Skew swimmingness rate (0 = fixed skew, 100 = ATM anchored).
        Ref (float): Reference forward price.
        ATM (float): ATM forward used in floating skew calculation.

    Returns:
        strikes (array): The original strike array.
        vols (array): Computed implied volatilities corresponding to the strikes.
    """
    # Convert SSR to a fractional weight (0 to 1)
    ssr_frac = SSR / 100.0

    # Compute effective forward (F_eff) based on SSR blend between Ref and ATM
    if SSR == 0:
        F_eff = Ref
    elif SSR == 100:
        F_eff = ATM
    else:
        F_eff = (1 - ssr_frac) * Ref + ssr_frac * ATM

    # Central volatility adjusted by forward deviation from reference
    vc = vr - VCR * ssr_frac * (F_eff - Ref) / Ref

    # Central slope adjusted by forward deviation
    sc = sr - SCR * ssr_frac * (F_eff - Ref) / Ref

    # Convert strike to log-moneyness x = ln(K/F_eff)
    x = np.log(strikes / F_eff)
    vols = np.zeros_like(x)

    # Precompute Hermite endpoints
    xL0, xL1 = dc - dsm, dc
    yL0 = vc + sc * xL0 + pc * xL0 ** 2
    yL1 = vc + sc * xL1 + pc * xL1 ** 2
    dyL0 = sc + 2 * pc * xL0
    dyL1 = sc + 2 * pc * xL1

    xR0, xR1 = uc, uc + usm
    yR0 = vc + sc * xR0 + cc * xR0 ** 2
    yR1 = vc + sc * xR1 + cc * xR1 ** 2
    dyR0 = sc + 2 * cc * xR0
    dyR1 = sc + 2 * cc * xR1

    for i, xi in enumerate(x):
        if xi < xL0:
            # Extrapolate left of smoothing using tangent at xL0
            vols[i] = yL0 + dyL0 * (xi - xL0)

        elif xL0 <= xi < xL1:
            # Hermite interpolation in down smoothing zone
            t = (xi - xL0) / (xL1 - xL0)
            h00 = 2 * t**3 - 3 * t**2 + 1
            h10 = t**3 - 2 * t**2 + t
            h01 = -2 * t**3 + 3 * t**2
            h11 = t**3 - t**2
            vols[i] = h00 * yL0 + h10 * (xL1 - xL0) * dyL0 + h01 * yL1 + h11 * (xL1 - xL0) * dyL1

        elif dc <= xi < 0:
            # Put wing
            vols[i] = vc + sc * xi + pc * xi ** 2

        elif 0 <= xi <= uc:
            # Call wing
            vols[i] = vc + sc * xi + cc * xi ** 2

        elif xR0 < xi <= xR1:
            # Hermite interpolation in up smoothing zone
            t = (xi - xR0) / (xR1 - xR0)
            h00 = 2 * t**3 - 3 * t**2 + 1
            h10 = t**3 - 2 * t**2 + t
            h01 = -2 * t**3 + 3 * t**2
            h11 = t**3 - t**2
            vols[i] = h00 * yR0 + h10 * (xR1 - xR0) * dyR0 + h01 * yR1 + h11 * (xR1 - xR0) * dyR1

        elif xi > xR1:
            # Extrapolate right of smoothing using tangent at xR1
            vols[i] = yR1 + dyR1 * (xi - xR1)

    return strikes, vols

In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, FloatText, HBox, VBox, interactive_output, Layout

# Define float slider + input box pair
def float_slider_text(label, min_val, max_val, step, value):
    slider = FloatSlider(min=min_val, max=max_val, step=step, value=value,
                         description=label, continuous_update=False, layout=Layout(width='60%'))
    text = FloatText(value=value, layout=Layout(width='80px'))
    # Link both ways
    def link_slider_text(*args): slider.value = text.value
    def link_text_slider(*args): text.value = slider.value
    slider.observe(link_text_slider, 'value')
    text.observe(link_slider_text, 'value')
    return slider, text

# Plot function
def interactive_wing_plot(
    vr, sr, pc, cc, dc, uc, dsm, usm, VCR, SCR, SSR,
    F, Ref, ATM
):
    strikes = np.linspace(30, 180, 200)
    strikes_out, vols_out = wing_vol_curve_with_smoothing(
        strikes=strikes,
        F=F, vr=vr, sr=sr, pc=pc, cc=cc,
        dc=dc, uc=uc, dsm=dsm, usm=usm,
        VCR=VCR, SCR=SCR, SSR=SSR,
        Ref=Ref, ATM=ATM
    )

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=strikes_out, y=vols_out, mode='lines', name="Implied Vol"))
    fig.update_layout(
        title="Interactive Wing Model Volatility Curve",
        xaxis_title="Strike Price",
        yaxis_title="Implied Volatility",
        template="plotly_white",
        height=500
    )
    fig.show()

# Define all widgets
params = {
    'vr': float_slider_text("Vol Ref", 0.05, 0.5, 0.01, 0.20),
    'sr': float_slider_text("Slope", -2.0, 2.0, 0.1, 0.0),
    'pc': float_slider_text("Put Curv", 0.0, 5.0, 0.1, 2.0),
    'cc': float_slider_text("Call Curv", 0.0, 5.0, 0.1, 1.0),
    'dc': float_slider_text("Down Cut", -1.0, 0.0, 0.05, -0.2),
    'uc': float_slider_text("Up Cut", 0.0, 1.0, 0.05, 0.2),
    'dsm': float_slider_text("Down Sm", 0.0, 2.0, 0.1, 0.5),
    'usm': float_slider_text("Up Sm", 0.0, 2.0, 0.1, 0.5),
    'VCR': float_slider_text("VCR", -2.0, 2.0, 0.1, 0.0),
    'SCR': float_slider_text("SCR", -2.0, 2.0, 0.1, 0.0),
    'SSR': float_slider_text("SSR", 0, 100, 1, 50),
    'F': float_slider_text("Forward F", 80, 120, 1, 100),
    'Ref': float_slider_text("Reference", 80, 120, 1, 90),
    'ATM': float_slider_text("ATM", 80, 120, 1, 110),
}

# Organize layout
widget_boxes = [HBox([s, t]) for s, t in params.values()]
ui = VBox(widget_boxes)

# Create plot output
interactive_plot = interactive_output(interactive_wing_plot, {k: v[0] for k, v in params.items()})

# Display in notebook
display(ui, interactive_plot)


Output()